In [22]:
import requests
import pandas as pd

In [23]:
ruta_archivo = r"../data/processed/datos_filtradosv1.csv"
# Cargamos los datos del CSV a un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

In [24]:
# Seleccionamos las columnas que queremos mantener
df = df[['Departamento',
         'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion',
         'Es Grupo', 'Es Pyme', 'Liquidación',
         'Obligación Ambiental',
         'Valor del Contrato', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado',
         'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos Propios'
         ]]

df.info()

#### Conversion de las fechas a DateTime

In [25]:
columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion']

# Iterar sobre las columnas y convertirlas al tipo de dato 'date'
for columna in columnas_fecha:
    df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
    df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#### Filtrar por el estado del contrato

In [26]:
estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]

# Filtrar las filas del dataframe basándote en la columna "Estado Contrato"
df_filtradoNoEjecucion = df[df["Estado Contrato"].isin(estados_contrato)]
df_filtradoNoEjecucion

#### Filtrar por año del contrato

In [27]:
df_filtradoAnio = df_filtradoNoEjecucion[df_filtradoNoEjecucion['Fecha de Inicio del Contrato'].dt.year >= 2019]
df_filtradoAnio

In [28]:
# df_filtradoAnio = df_filtradoAnio.drop_duplicates()
# df_filtradoAnio.info()

In [29]:
df_filtradoAnio.shape

In [30]:
df_filtradoAnio["EsServicioPublico"] = df_filtradoAnio['Sector'] == 'Servicio Público'
df_filtradoAnio = df_filtradoAnio.drop('Sector', axis=1)

df_filtradoAnio["EsPrestacionServicios"] = df_filtradoAnio['Tipo de Contrato'] == 'Prestación de servicios'
df_filtradoAnio = df_filtradoAnio.drop('Tipo de Contrato', axis=1)

df_filtradoAnio["EsGrupo"] = df_filtradoAnio['Es Grupo'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Grupo', axis=1)

df_filtradoAnio["EsPyme"] = df_filtradoAnio['Es Pyme'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Es Pyme', axis=1)

df_filtradoAnio["EstaLiquidado"] = df_filtradoAnio['Liquidación'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Liquidación', axis=1)

df_filtradoAnio["EsObligacionAmbiental"] = df_filtradoAnio['Obligación Ambiental'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('Obligación Ambiental', axis=1)


df_filtradoAnio["Es PostConflicto"] = df_filtradoAnio['EsPostConflicto'] == 'Si'
df_filtradoAnio = df_filtradoAnio.drop('EsPostConflicto', axis=1)

df_filtradoAnio["EsRecursosPropios"] = df_filtradoAnio['Origen de los Recursos'] == 'Recursos Propios'
df_filtradoAnio = df_filtradoAnio.drop('Origen de los Recursos', axis=1)


df_filtradoAnio = df_filtradoAnio.drop_duplicates()
df_filtradoAnio.info()

df_filtradoAnio

In [31]:
# df_filtradoAnio.to_csv('datos_filtrados_20230719.csv', index=False)

In [32]:
muestra_aleatoria = df_filtradoAnio.sample(frac=0.4)
muestra_aleatoria.shape

In [33]:
from ydata_profiling import ProfileReport
profile = ProfileReport(muestra_aleatoria, title="Pandas Profiling Report")
profile.to_file("secopFiltro2021muestra_profile_20230719_1923.html")

In [34]:
# Aplicar one-hot encoding
df_one_hot = pd.get_dummies(df_filtradoAnio['Estado Contrato'])

# Combinar el DataFrame original con las nuevas columnas
df_final = pd.concat([df_filtradoAnio, df_one_hot], axis=1)

# Eliminar la columna original "Estado Contrato" si deseas
df_final = df_final.drop('Estado Contrato', axis=1)

df_final

In [35]:
estado_contrato = df_final["Cerrado"].copy()
contratos_data = df_final.drop(["terminado","cedido","Suspendido","Cerrado"],axis=1)

In [36]:
# Calculate test and validation set size:
original_count = len(df_final)
training_size = 0.60 # 60% of records
test_size = (1 - training_size) / 2


training_count = int(original_count * training_size)
test_count = int(original_count * test_size)
validation_count = original_count - training_count - test_count

print(training_count, test_count, validation_count, original_count)

In [37]:
from sklearn.model_selection import train_test_split

train_x, rest_x, train_y, rest_y = train_test_split(contratos_data, estado_contrato, train_size=training_count)
test_x, validate_x, test_y, validate_y = train_test_split(rest_x, rest_y, train_size=test_count)

print(len(train_x), len(test_x), len(validate_x))

# Pipeline

In [38]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline

In [39]:
one_hot_encoding = ColumnTransformer([
    (
        'one_hot_encode',
        OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
        [
            "Orden",
            "Entidad Centralizada",
            "Modalidad de Contratacion",
            "Destino Gasto",
            "Género Representante Legal"
        ]
    )
])

In [40]:
binarizer = ColumnTransformer([
    (
        'binarizer',
        Binarizer(),
        [
            "Sector"
        ]
    )
])

one_hot_binarized = Pipeline([
    ("binarizer", binarizer),
    ("one_hot_encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
])